In [2]:
import os
import csv
import math

import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
import os
import csv
import math
import itertools
import pickle

import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
import os
import csv
import math
import itertools
import pickle

import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from scipy import sparse
import networkx as nx

from mbi import (
    Dataset,
    FactoredInference,
    Domain,
    LocalInference,
    MixtureInference,
    PublicInference,
)
import os
import csv
import math
import itertools
import pickle

import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from fairness_checker import *

In [3]:
# source: https://www.kaggle.com/datasets/danofer/compass
df = pd.read_csv("./data/compas_og.csv")
df.head()

# source: https://www.kaggle.com/code/innocentsky/logistics-regression-compas-recidivism-racial-bias
columns_to_drop = ['id', 'first', 'last', 'in_custody', 'vr_charge_desc', 'r_case_number', 'vr_charge_degree',
                   'c_offense_date', 'c_case_number', 'r_offense_date', 'juv_other_count', 'end', 'event',
                   'screening_date', 'start', 'juv_misd_count', 'juv_fel_count', 'r_days_from_arrest',
                   'r_charge_degree', 'days_b_screening_arrest', 'vr_case_number', 'priors_count.1', 'r_jail_out',
                   'c_arrest_date', 'r_charge_desc', 'r_jail_in', 'violent_recid', 'decile_score.1',
                   'vr_offense_date', 'out_custody']
df.drop(columns=columns_to_drop, axis=1, inplace=True, errors='ignore')

df['c_jail_in'] = pd.to_datetime(df['c_jail_in'], dayfirst=True)
df['c_jail_out'] = pd.to_datetime(df['c_jail_out'], dayfirst=True)
df['compas_screening_date'] = pd.to_datetime(df['compas_screening_date'], dayfirst=True)
df['v_screening_date'] = pd.to_datetime(df['v_screening_date'], dayfirst=True)
df['days_in_jail'] = abs((df['c_jail_out'] - df['c_jail_in']).dt.days)

numeric_cols = ['c_days_from_compas', 'v_decile_score',]
for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())
categorical_cols = ['c_charge_degree', 'score_text', 'v_score_text', 'c_jail_in', 'c_jail_out', 'c_charge_desc', 'days_in_jail']
for col in categorical_cols:
    df[col] = df[col].fillna(df[col].mode()[0])

df_small = df[["age_cat", "sex", "race", "is_recid"]].copy()

columns_to_encode = ['age_cat', 'sex', 'race']
le = preprocessing.LabelEncoder()
for column in columns_to_encode:
    df_small[column] = le.fit_transform(df_small[column])

df_small.to_csv('./data/compas_processed_small.csv', index=False)
df_small.head()

,age_cat,sex,race,is_recid
0,1,1,5,0
1,1,1,5,0
2,0,1,2,-1
3,0,1,0,1
4,2,1,0,1


In [5]:
dataset_small = pd.read_csv("./data/compas_processed_small.csv")

X_small = dataset_small[["age_cat", "sex", "race"]].copy()
Y_small = dataset_small['is_recid']

X_train_small, X_test_small, Y_train_small, Y_test_small = train_test_split(X_small, Y_small, test_size=0.3,
                                                    stratify=X_small[['race']], random_state=42)

clf_small = RandomForestClassifier(
        max_depth=None,
        min_samples_split=2,
        n_estimators=50
    )

clf_small.fit(X_train_small, Y_train_small)

test_accuracy = clf_small.score(X_test_small, Y_test_small)
print(f"Test Accuracy: {test_accuracy}")

with open('./model/compas_RF_small.pkl', 'wb') as f:
    pickle.dump(clf_small, f)

Test Accuracy: 0.5659690627843494


In [8]:
data_small = Dataset.load("./data/compas_processed_small.csv", "./data/compas_processed_small.json")
domain_small = data_small.domain
total = data_small.df.shape[0]

# fully connected graph
fields = list(data_small.domain)
cliques = [
    (fields[i], fields[j]) for i in range(len(fields)) for j in range(i+1, len(fields))
]

# spend half of privacy budget to measure all 1 way marginals
np.random.seed(0)

epsilon = 1.0
epsilon_split = epsilon / (len(data_small.domain) + len(cliques))
sigma = 2.0 / epsilon_split

measurements = []
for col in data_small.domain:
    x = data_small.project(col).datavector()
    y = x + np.random.laplace(loc=0, scale=sigma, size=x.size)
    I = sparse.eye(x.size)
    measurements.append((I, y, sigma, (col,)))

# spend half of privacy budget to measure some more 2 and 3 way marginals
for cl in cliques:
    x = data_small.project(cl).datavector()
    y = x + np.random.laplace(loc=0, scale=sigma, size=x.size)
    I = sparse.eye(x.size)
    measurements.append((I, y, sigma, cl))

engine = FactoredInference(domain_small, log=True, iters=500)
model = engine.estimate(measurements, total=total)

# Save the model to a file
with open('./model/compas_synth_small.pkl', 'wb') as f:
    pickle.dump(model, f)

Total clique size: 108
iteration		time		l1_loss		l2_loss		feasibility
0.00		0.00		7791.95		634057.27		0.00
50.00		0.22		255.34		753.78		0.00
100.00		0.45		227.04		695.66		0.00
150.00		0.69		223.61		691.56		0.00
200.00		0.90		222.11		689.75		0.00
250.00		1.12		221.16		688.47		0.00
300.00		1.34		220.54		687.47		0.00
350.00		1.55		220.07		686.65		0.00
400.00		1.78		219.70		685.99		0.00
450.00		1.99		219.39		685.47		0.00


In [12]:
# To load the model back
with open('./model/compas_RF_small.pkl', 'rb') as f:
    clf = pickle.load(f)
with open('./model/compas_synth_small.pkl', 'rb') as f:
    model = pickle.load(f)

class model_wrapper:
    def __init__(self, model):
        self.model = model

    def predict(self, file):
        df = pd.read_csv(file)
        X = self.preprocessing(df)

        return self.model.predict(X)

    def preprocessing(self, df):
        df = df[["age_cat", "sex", "race"]].copy()

        return df

trained = model_wrapper(clf)

iter = 10

disparate_impact = []
demographic_parity = []
equalized_odds_1, equalized_odds_2 = [], []
equal_opportunity = []
accuracy_eqaulity = []
predictive_parity = []
equal_calibration = []
conditional_statistical_parity = []
predictive_equality = []
conditional_use_accuracy_equality_1, conditional_use_accuracy_equality_2 = [], []
positive_balance = []
negative_balance = []
mean_difference = []

for _ in range(iter):
    synth = model.synthetic_data(rows=70000)
    sdf = synth.df
    sdf.to_csv('./tmp/synth.csv', index=False)

    c = fairness_model_checker("./tmp/synth.csv", verbose=False)

    privileged_predicate = lambda row: row['sex'] != '0'
    positive_predicate = lambda Y: Y == 1
    truth_predicate = lambda row: row['is_recid'] == '1'

    disparate_impact               .append( c.disparate_impact(0.8, trained, privileged_predicate, positive_predicate, value=True) )
    demographic_parity             .append( c.demographic_parity(0.2, trained, privileged_predicate, positive_predicate, value=True) )
    (x1, x2) = c.equalized_odds(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
    equalized_odds_1 .append( x1 )
    equalized_odds_2 .append( x2 )
    equal_opportunity              .append( c.equal_opportunity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    accuracy_eqaulity              .append( c.accuracy_eqaulity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    predictive_parity              .append( c.predictive_parity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    # equal_calibration (n/a)
    conditional_statistical_parity .append( c.conditional_statistical_parity(0.2, trained, privileged_predicate, positive_predicate, lambda x: (lambda row: row['race'] == x), ('4',), value=True) )
    predictive_equality            .append( c.predictive_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True) )
    (y1, y2) = c.conditional_use_accuracy_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
    conditional_use_accuracy_equality_1 .append( y1 )
    conditional_use_accuracy_equality_2 .append( y2 )
    # positive_balance (n/a)
    # negative_balance (n/a)
    mean_difference .append( c.mean_difference(0.2,trained, privileged_predicate, positive_predicate, value=True) )

og_disparate_impact = 0
og_demographic_parity = 0
og_equalized_odds_1, og_equalized_odds_2 = 0, 0
og_equal_opportunity = 0
og_accuracy_eqaulity = 0
og_predictive_parity = 0
og_equal_calibration = 0
og_conditional_statistical_parity = 0
og_predictive_equality = 0
og_conditional_use_accuracy_equality_1, og_conditional_use_accuracy_equality_2 = 0, 0
og_positive_balance = 0
og_negative_balance = 0
og_mean_difference = 0

cog = fairness_model_checker("./data/compas_processed_small.csv", verbose=False)

privileged_predicate = lambda row: row['sex'] == '0'
positive_predicate = lambda Y: Y == 1
truth_predicate = lambda row: row['is_recid'] == '1'

og_disparate_impact = cog.disparate_impact(0.8, trained, privileged_predicate, positive_predicate, value=True)
og_demographic_parity = cog.demographic_parity(0.2, trained, privileged_predicate, positive_predicate, value=True)
(og_equalized_odds_1, og_equalized_odds_2) = cog.equalized_odds(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
og_equal_opportunity = cog.equal_opportunity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
og_accuracy_eqaulity = cog.accuracy_eqaulity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
og_predictive_parity = cog.predictive_parity(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
# equal_calibration (n/a)
og_conditional_statistical_parity = cog.conditional_statistical_parity(0.2, trained, privileged_predicate, positive_predicate, lambda x: (lambda row: row['race'] == x), ('4',), value=True)
og_predictive_equality = cog.predictive_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
(og_conditional_use_accuracy_equality_1, og_conditional_use_accuracy_equality_2) = cog.conditional_use_accuracy_equality(0.2, trained, privileged_predicate, positive_predicate, truth_predicate, value=True)
# positive_balance (n/a)
# negative_balance (n/a)
og_mean_difference = cog.mean_difference(0.2, trained, privileged_predicate, positive_predicate, value=True)

print("name               ", "og          ", "synth       ", "diff        ", "std")
print("demographic_parity ", f"{og_demographic_parity:.10f}", f"{np.mean(np.array(demographic_parity)):.10f}", f"{abs(og_demographic_parity - np.mean(np.array(demographic_parity))):.10f}", np.std(np.array(demographic_parity)))
print("accuracy_eqaulity  ", f"{og_accuracy_eqaulity:.10f}", f"{np.mean(np.array(accuracy_eqaulity)):.10f}", f"{abs(og_accuracy_eqaulity - np.mean(np.array(accuracy_eqaulity))):.10f}", np.std(np.array(accuracy_eqaulity)))
print("equalized_odds_1   ", f"{og_equalized_odds_1:.10f}", f"{np.mean(np.array(equalized_odds_1)):.10f}", f"{abs(og_equalized_odds_1 - np.mean(np.array(equalized_odds_1))):.10f}", np.std(np.array(equalized_odds_1)))
print("equalized_odds_2   ", f"{og_equalized_odds_2:.10f}", f"{np.mean(np.array(equalized_odds_2)):.10f}", f"{abs(og_equalized_odds_2 - np.mean(np.array(equalized_odds_2))):.10f}", np.std(np.array(equalized_odds_2)))
print("accuracy_equality_1", f"{og_conditional_use_accuracy_equality_1:.10f}", f"{np.mean(np.array(conditional_use_accuracy_equality_1)):.10f}", f"{abs(og_conditional_use_accuracy_equality_1 - np.mean(np.array(conditional_use_accuracy_equality_1))):.10f}", np.std(np.array(conditional_use_accuracy_equality_1)))
print("accuracy_equality_2", f"{og_conditional_use_accuracy_equality_2:.10f}", f"{np.mean(np.array(conditional_use_accuracy_equality_2)):.10f}", f"{abs(og_conditional_use_accuracy_equality_2 - np.mean(np.array(conditional_use_accuracy_equality_2))):.10f}", np.std(np.array(conditional_use_accuracy_equality_2)))
print("mean_difference    ", f"{og_mean_difference:.10f}", f"{np.mean(np.array(mean_difference)):.10f}", f"{abs(og_mean_difference - np.mean(np.array(mean_difference))):.10f}", np.std(np.array(mean_difference)))

print(
abs(og_demographic_parity - np.mean(np.array(demographic_parity)))+
abs(og_accuracy_eqaulity - np.mean(np.array(accuracy_eqaulity)))+
abs(og_equalized_odds_1 - np.mean(np.array(equalized_odds_1)))+
abs(og_equalized_odds_2 - np.mean(np.array(equalized_odds_2)))+
abs(og_conditional_use_accuracy_equality_1 - np.mean(np.array(conditional_use_accuracy_equality_1)))+
abs(og_conditional_use_accuracy_equality_2 - np.mean(np.array(conditional_use_accuracy_equality_2)))+
abs(og_mean_difference - np.mean(np.array(mean_difference)))
)

name                og           synth        diff         std
demographic_parity  0.5486594482 0.5511897117 0.0025302635 7.064220366964755e-05
accuracy_eqaulity   0.0521064440 0.0436973164 0.0084091275 8.964695473530937e-05
equalized_odds_1    0.4656165590 0.4764343778 0.0108178187 0.0001308542902427566
equalized_odds_2    0.6367930420 0.6282030090 0.0085900330 8.481207977335857e-05
accuracy_equality_1 0.4908039158 0.1287840150 0.3620199008 0.07389679712419607
accuracy_equality_2 0.1602493012 0.1179371391 0.0423121621 0.011589581430400644
mean_difference     0.4511876964 0.4516402045 0.0004525080 0.0011989953319592151
0.4351318136398984
